In [1]:
data_var = '2023-12-26'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
6795,2023-12-26,Eua Nba,00:30,Phoenix Suns,Dallas Mavericks,1.60,2.57,240.5,1.89,2.04,-4.5,1.97,1.97,fyVCZqDH,0.625000,0.389105,0.529101,0.490196,0.014105,0.4,0.6,NaN,NaN,177.444,50.505632,0.284629,255.386,56.018638,0.219349,187.958,238.350,127.68,305.28,0.0,0.0,0.0,0.0,0.328966,0.053978,0.000000,-1.93,-0.386,-1.554404,0.667661,0.6,-0.067661,-0.44,-0.088,-17.840909,0.550473,0.5,-0.050473
6796,2023-12-26,Eua Nba,21:00,Detroit Pistons,Brooklyn Nets,3.18,1.43,232.5,1.84,1.92,5.5,2.01,1.78,YRTGY3SN,0.314465,0.699301,0.543478,0.520833,0.013766,0.6,0.6,NaN,NaN,866.376,682.346663,0.787587,227.322,66.935512,0.294452,377.306,292.916,230.88,178.20,0.0,0.0,0.0,0.0,0.536849,0.030090,0.085823,-5.00,-1.000,-2.180000,0.352794,0.0,-0.352794,0.29,0.058,7.413793,0.399812,0.4,0.000188
6797,2023-12-26,Eua Nba,21:00,Washington Wizards,Orlando Magic,3.48,1.37,235.5,1.80,2.09,6.5,2.06,1.86,GjJLXNsU,0.287356,0.729927,0.555556,0.478469,0.017283,0.8,0.4,NaN,NaN,688.036,310.692637,0.451565,307.406,108.828736,0.354023,502.580,335.314,508.27,239.85,1.0,0.0,0.0,0.0,0.615256,0.105430,0.072154,-1.29,-0.258,-9.612403,0.331658,0.2,-0.131658,-2.95,-0.590,-0.627119,0.410824,0.4,-0.010824
6798,2023-12-26,Eua Nba,22:00,Chicago Bulls,Atlanta Hawks,1.84,2.12,233.5,1.96,1.98,-3.5,2.10,1.65,hCAmTsjo,0.543478,0.471698,0.510204,0.505051,0.015176,0.6,0.6,NaN,NaN,344.444,114.944343,0.333710,277.004,27.598901,0.099634,273.284,315.330,192.66,314.90,0.0,0.0,0.0,0.0,0.099995,0.007179,0.169706,3.43,0.686,1.224490,0.457893,0.6,0.142107,-0.42,-0.084,-13.333333,0.439376,0.4,-0.039376
6799,2023-12-26,Eua Nba,22:00,Houston Rockets,Indiana Pacers,1.72,2.30,235.5,1.84,2.03,-3.5,2.02,1.90,6FbViRSo,0.581395,0.434783,0.543478,0.492611,0.016178,0.4,0.6,NaN,NaN,294.884,95.751321,0.324708,234.802,82.783831,0.352569,182.430,274.810,174.46,263.68,0.0,0.0,0.0,0.0,0.204041,0.069432,0.043292,1.30,0.260,2.769231,0.570023,0.9,0.329977,-3.58,-0.716,-1.815642,0.410366,0.4,-0.010366
6800,2023-12-26,Eua Nba,22:00,New Orleans Pelicans,Memphis Grizzlies,1.58,2.62,227.5,1.80,2.09,-5.5,2.04,1.88,p6cZjorh,0.632911,0.381679,0.555556,0.478469,0.014591,0.4,0.6,NaN,NaN,179.110,34.832555,0.194476,302.196,110.750750,0.366486,178.756,330.380,149.76,238.75,0.0,0.0,0.0,0.0,0.350186,0.105430,0.057723,-0.17,-0.034,-17.058824,0.550628,0.7,0.149372,2.39,0.478,3.389121,0.353638,0.5,0.146362
6801,2023-12-26,Eua Nba,22:00,Oklahoma City Thunder,Minnesota Timberwolves,1.61,2.54,224.5,1.81,1.95,-5.5,2.04,1.72,2shvj5cb,0.621118,0.393701,0.552486,0.512821,0.014819,0.8,0.8,NaN,NaN,232.910,76.913554,0.330229,212.148,39.450413,0.185957,185.388,215.370,196.80,259.60,0.0,0.0,0.0,0.0,0.316920,0.052657,0.120359,0.57,0.114,5.350877,0.678209,0.8,0.121791,2.12,0.424,3.632075,0.557519,0.8,0.242481
6802,2023-12-26,Eua Nba,22:00,San Antonio Spurs,Utah Jazz,2.30,1.72,240.5,1.81,2.07,1.5,2.10,1.84,tYjrkPC4,0.434783,0.581395,0.552486,0.483092,0.016178,1.0,0.6,NaN,NaN,637.654,633.128100,0.992902,358.966,101.692396,0.283293,365.638,364.578,368.50,371.70,1.0,0.0,0.0,0.0,0.204041,0.094767,0.093324,-1.65,-0.330,-3.939394,0.327056,0.1,-0.227056,2.55,0.510,1.411765,0.334881,0.3,-0.034881
6803,2023-12-26,Europa Eurocopa,15:30,JL Bourg,Gran Canaria,1.80,1.92,162.5,1.84,1.86,-2.5,2.09,1.66,pprHl6OH,0.555556,0.520833,0.543478,0.537634,0.076389,0.2,0.0,NaN,NaN,149.298,34.248565,0.229397,124.170,34.582498,0.278509,129.052,0.000,166.00,161.37,0.0,0.0,0.0,0.0,0.045620,0.007644,0.16216

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:30,Eua Nba,Phoenix Suns,Dallas Mavericks,240.5,1.89,1.0000,Over
2,21:00,Eua Nba,Washington Wizards,Orlando Magic,235.5,1.80,0.9999,Over
4,22:00,Eua Nba,Houston Rockets,Indiana Pacers,235.5,1.84,0.9999,Over
5,22:00,Eua Nba,New Orleans Pelicans,Memphis Grizzlies,227.5,1.80,1.0000,Over
7,22:00,Eua Nba,San Antonio Spurs,Utah Jazz,240.5,1.81,0.9999,Over
